## Librerias

In [0]:
!pip install spacy-langdetect
from textblob import TextBlob
import numpy as np
import pandas as pd
import spacy
from spacy_langdetect import LanguageDetector


     |████████████████████████████████| 1.0MB 8.2MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993459 sha256=b1fb76dad5c3cd4913c711f21518a8a257139202b53c0f704b1d76b4d0ccf3e8
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
ruta = "/content/gdrive/My Drive/KeepCoding/ProyectoFinal/"

## Carga del fichero. Reviews.csv

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path_csv = ruta + 'data/reviews.csv'

In [0]:
df = pd.read_csv(path_csv, sep =',', parse_dates=['date']) # cargamos fichero
df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,6369,29428,2010-03-14,84790,Nancy,Simon and Arturo have the ultimate location in...
1,6369,31018,2010-03-23,84338,David,Myself and Kristy originally planned on stayin...
2,6369,34694,2010-04-10,98655,Marion,We had a great time at Arturo and Simon's ! A ...
3,6369,37146,2010-04-21,109871,Kurt,I very much enjoyed the stay. \r\nIt's a wond...
4,6369,38168,2010-04-26,98901,Dennis,Arturo and Simon are polite and friendly hosts...


In [0]:
df.comments

0         Simon and Arturo have the ultimate location in...
1         Myself and Kristy originally planned on stayin...
2         We had a great time at Arturo and Simon's ! A ...
3         I very much enjoyed the stay.  \r\nIt's a wond...
4         Arturo and Simon are polite and friendly hosts...
                                ...                        
776364                 nice locations,muy hospital y amable
776365    Good location, friendly host that communicates...
776366       Comfy and cozy, but a bit far from everything.
776367    Victors place is perfect for anyone wanting to...
776368    A Clean room in a clean apartment, in a good a...
Name: comments, Length: 776369, dtype: object

In [0]:
#Convierto a string los comentarios
df.comments = df.comments.astype('str')

## Idioma 
Los comentarios están en varios idiomas. Voy a utilizar la libreria spacy para detectar en que idioma está cada comentarios y posteriormente quedarme solo con los que estén en inglés, ya que para el Sentimient Analisys voy a usar un a libreria que trabaja solo en inglés

https://pypi.org/project/spacy-langdetect/

In [0]:
#Voy a recorrer la columna comments y añado una nueva con el idioma
##NOTA: Esta instrucción tarda bastante, unas 3 horas
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)


df['idioma'] = df ['comments'].apply(lambda x: nlp(x)._.language['language'])

In [0]:
df.head(10)

,listing_id,id,date,reviewer_id,reviewer_name,comments,idioma
0,6369,29428,2010-03-14,84790,Nancy,Simon and Arturo have the ultimate location in...,en
1,6369,31018,2010-03-23,84338,David,Myself and Kristy originally planned on stayin...,en
2,6369,34694,2010-04-10,98655,Marion,We had a great time at Arturo and Simon's ! A ...,en
3,6369,37146,2010-04-21,109871,Kurt,I very much enjoyed the stay. \r\nIt's a wond...,en
4,6369,38168,2010-04-26,98901,Dennis,Arturo and Simon are polite and friendly hosts...,en
5,6369,41548,2010-05-10,95579,Deborah,Perfect airbnb experience as all others say. L...,en
6,6369,42550,2010-05-15,106586,Nina,"Great host, great apartment! I had the bath to...",en
7,6369,45278,2010-05-23,112662,Bo,We had a wonderful time in Madrid and the stay...,en
8,6369,45618,2010-05-24,18688,Charde,We only stayed here for one night unfortunatel...,en
9,6369,57074,2010-06-25,134713,Y. S.,Great room in non-touristy location. Convenie...,en


In [0]:
#Veamos los idiomas y cuantos hay de cada
dato = df.idioma.value_counts()
pd.set_option('max_rows', len(dato)) # Para ver todos
dato

en         375972
es         277961
fr          46201
pt          16911
it          15790
de          10711
ko           6467
zh-cn        5118
ru           3841
nl           3680
ro           2019
UNKNOWN      1792
ca           1705
pl            845
da            783
so            644
tl            638
af            602
ja            527
sv            470
fi            437
no            345
cs            301
hu            287
sk            279
id            264
tr            240
cy            207
sl            189
el            180
vi            159
he            131
hr            129
sw            113
lt            113
et             72
zh-tw          60
bg             49
uk             34
lv             33
ar             31
mk             15
sq             13
th              9
fa              1
ur              1
Name: idioma, dtype: int64

Los mayoritarios son el inglés y el español. Me quedo solo con los que se han detectado como están en inglés

In [0]:
#Me quedo con el subconjunto de los que están en ingles

df = df [df.idioma == 'en']

In [0]:
dato = df.idioma.value_counts()
pd.set_option('max_rows', len(dato)) # Para ver todos
dato

en    375972
Name: idioma, dtype: int64

## Sentimental Analysis
Voy a utilizar la librería TextBlob

https://github.com/sloria/textblob

https://textblob.readthedocs.io/en/latest/quickstart.html#quickstart

TextBlob es una librería Python (2 y 3) para procesar datos de texto. Proporciona una API consistente para tareas comunes de procesamiento de lenguaje natural como etiquetado de parte del habla, extracción de frases sustantivas, análisis de sentimientos, y más.

En concreto yo la voy a utilizar para el análisis del sentimiento. Con TextBlob, obtengo una métrica de polaridad y subjetividad. La polaridad es el sentimiento mismo, que va de -1 a +1. La subjetividad es una medida del sentimiento siendo objetivo a subjetivo, y va de 0 a 1. 

In [0]:
#Utilizo TexBlob sobre la columna comments y me creo una nueva columna textblob con el resultado
df['textblob'] = df ['comments'].apply(lambda x: TextBlob(x))

In [0]:
#Creo una nueva columna en el dataset "sentiment" y aquí añado la metríca de polaridad recorriendo 
#la columna textblob
df['sentiment'] = df ['textblob'].apply(lambda x: x.sentiment.polarity )

In [0]:
df.head(50)

,listing_id,id,date,reviewer_id,reviewer_name,comments,idioma,textbox,sentiment
0,6369,29428,2010-03-14,84790,Nancy,Simon and Arturo have the ultimate location in...,en,"(S, i, m, o, n, , a, n, d, , A, r, t, u, r, ...",0.214524


In [0]:
#Guardo el resultado obtenido por si pudiera hacer falta 
path_sentiment = ruta + 'data/reviews_sentiment.csv'
reviews_sentiment = df.to_csv(path_sentiment, header=True)

In [0]:
#Cargo de nuevo para continuar
path_csv = ruta + 'data/reviews_sentiment.csv'


In [0]:
df = pd.read_csv(path_csv, sep =',', parse_dates=['date']) # cargamos fichero
df.head(5) 

,Unnamed: 0,listing_id,id,date,reviewer_id,reviewer_name,comments,idioma,textbox,sentiment
0,0,6369,29428,2010-03-14,84790,Nancy,Simon and Arturo have the ultimate location in...,en,Simon and Arturo have the ultimate location in...,0.214524
1,1,6369,31018,2010-03-23,84338,David,Myself and Kristy originally planned on stayin...,en,Myself and Kristy originally planned on stayin...,0.346267
2,2,6369,34694,2010-04-10,98655,Marion,We had a great time at Arturo and Simon's ! A ...,en,We had a great time at Arturo and Simon's ! A ...,0.687500
3,3,6369,37146,2010-04-21,109871,Kurt,I very much enjoyed the stay. \r\nIt's a wond...,en,I very much enjoyed the stay. \r\nIt's a wond...,0.620238
4,4,6369,38168,2010-04-26,98901,Dennis,Arturo and Simon are polite and friendly hosts...,en,Arturo and Simon are polite and friendly hosts...,0.664167


Tenemos varios reviews por cada id_listings y por tanto varios sentiments. Lo que nos interesa es quedarnos con la media para cada uno de los id_listings

In [0]:
#Agrupo por listing_id y hago la media de sentiment
df['listing_id'] = df['listing_id'].astype('category')
id_sentiment = df.groupby('listing_id')['sentiment'].mean()
id_sentiment

listing_id
6369        0.392050
21853       0.367534
24805       0.680972
24836       0.383789
26825       0.366734
              ...   
39789020    0.432245
39882978    0.358333
39895614   -0.050000
39912805    0.500000
39916688    0.483333
Name: sentiment, Length: 15021, dtype: float64

In [0]:
#Convierto el resultado en un dataframe
df_sentiment = id_sentiment.to_frame ()
df_sentiment.head(10)

,sentiment
listing_id,
6369,0.392050
21853,0.367534
24805,0.680972
24836,0.383789
26825,0.366734
30320,0.367101
30959,0.340729
40916,0.334776
59923,0.362881


In [0]:
#Me interesa que listing_id sea una columna y no el indice de cara después a cruzarlo con la tabla 
#de listings
df_sentiment.reset_index (level=0, inplace=True)
df_sentiment.head(10)

,listing_id,sentiment
0,6369,0.392050
1,21853,0.367534
2,24805,0.680972
3,24836,0.383789
4,26825,0.366734
5,30320,0.367101
6,30959,0.340729
7,40916,0.334776
8,59923,0.362881
9,62423,0.481477


In [0]:
#Guardo el df_sentiment resultante
path_sentiment = ruta + 'data/idsentiment.csv'
idsentiment = df_sentiment.to_csv(path_sentiment, header=True)

## Unión con el dataset listings.csv

Lo siguiente sería unir este dataframe de sentimientos con el listado de viviendas de airbnb (listings.csv). La columna para unir sería la del listing_id

In [0]:
#Cargo el fichero listings.csv
path_csv = ruta + 'data/listings.csv'

In [0]:
house_data1 = pd.read_csv(path_csv, sep =',', decimal='.') # cargamos fichero
print(f"El dataset original contiene {len(house_data1)} viviendas Airbnb con {len(house_data1.columns)} características")
pd.set_option('display.max_columns', len(house_data1)) # Para ver todas las columnas
house_data1.head(5)  

El dataset original contiene 20539 viviendas Airbnb con 106 características


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6369,https://www.airbnb.com/rooms/6369,20191109094951,2019-11-09,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,BETTER THAN A HOTEL.Upscale neighboorhood (Met...,Excellent connection with the AIRPORT and EXHI...,none,Nice and definitely non touristic neighborhoo...,If you are a group/family 3 or 4 people we off...,Excelent public transport and easy Access to m...,"Full use of living room, kitchen (with dishwas...","English, spanish, german, russian, some french...",NaN,NaN,NaN,https://a0.muscache.com/im/pictures/683224/4cc...,NaN,13660,https://www.airbnb.com/users/show/13660,Simon,2009-04-16,"Madrid, Madrid, Spain","Gay couple, heterofriendly, enjoy having guest...",within a few hours,100%,NaN,f,https://a0.muscache.com/im/users/13660/profile...,https://a0.muscache.com/im/users/13660/profile...,Hispanoamérica,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Madrid, Comunidad de Madrid, Spain",Chamartín,Hispanoamérica,Chamartín,Madrid,Comunidad de Madrid,28016,Madrid,"Madrid, Spain",ES,Spain,40.45628,-3.67763,t,Apartment,Private room,2,1.0,1.0,0.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",172.0,$70.00,$350.00,$950.00,$0.00,$5.00,2,$15.00,1,365,1,1,365,365,1.0,365.0,today,t,29,59,89,89,2019-11-09,68,9,2010-03-14,2019-10-10,97.0,10.0,10.0,10.0,10.0,9.0,10.0,t,NaN,NaN,f,f,flexible,f,f,2,0,2,0,0.58
1,21853,https://www.airbnb.com/rooms/21853,20191109094951,2019-11-09,Bright and airy room,We have a quiet and sunny room with a good vie...,I am living in a nice flat near the centre of ...,We have a quiet and sunny room with a good vie...,none,We live in a leafy neighbourhood with plenty o...,We are a 15 min bus ride away from the Casa de...,The flat is near the centre of Madrid (15 minu...,There is fibre optic internet connection for y...,If I am at home and see each other around here...,Many thanks for not smoking at home.,NaN,NaN,https://a0.muscache.com/im/pictures/68483181/8...,NaN,83531,https://www.airbnb.com/users/show/83531,Abdel,2010-02-21,"Madrid, Madrid, Spain",EN-ES-FR\r\nEN\r\nHi everybody: I'm Abdel. I'm...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/83531/profile...,https://a0.muscache.com/im/users/83531/profile...,Aluche,2.0,2.0,"['em

A este dataset quiero unir el creado de sentiment según el id

Compruebo primero el tipo de dato id en ambos datasets

In [0]:
house_data1.id.dtypes

dtype('int64')

In [0]:
df_sentiment.listing_id.dtypes

CategoricalDtype(categories=[    6369,    21853,    24805,    24836,    26825,    30320,
                     30959,    40916,    59923,    62423,
                  ...
                  39739019, 39742771, 39752601, 39762796, 39788506, 39789020,
                  39882978, 39895614, 39912805, 39916688],
                 ordered=False)

In [0]:
#Voy a convertir de categoría a int64 para después poder unir por el id
df_sentiment['listing_id'] = df_sentiment['listing_id'].astype('int64')

In [0]:
df_sentiment.listing_id.dtypes

dtype('int64')

In [0]:
#Renombro la columna listing_id para que sea id y la pueda establecer como clave cuando una
df_sentiment.columns = ["id", "sentiment"]
df_sentiment.head(10)

,id,sentiment
0,6369,0.392050
1,21853,0.367534
2,24805,0.680972
3,24836,0.383789
4,26825,0.366734
5,30320,0.367101
6,30959,0.340729
7,40916,0.334776
8,59923,0.362881
9,62423,0.481477


In [0]:
#Uno los dos datasets usando como clave la columna id. Lo hago de tipo inner para quedarme con aquellos id
# que tengo en el dataset de aibnb y que tengan además review

house_data_sentimient = house_data1.join(df_sentiment.set_index('id'), on='id', how='inner')

In [0]:
print(f"El dataset nuevo contiene {len(house_data_sentimient)} viviendas Airbnb con {len(house_data_sentimient.columns)} características")
pd.set_option('display.max_columns', len(house_data_sentimient)) # Para ver todas las columnas
house_data_sentimient.head(5)  

El dataset nuevo contiene 15021 viviendas Airbnb con 107 características


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,sentiment
0,6369,https://www.airbnb.com/rooms/6369,20191109094951,2019-11-09,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,BETTER THAN A HOTEL.Upscale neighboorhood (Met...,Excellent connection with the AIRPORT and EXHI...,none,Nice and definitely non touristic neighborhoo...,If you are a group/family 3 or 4 people we off...,Excelent public transport and easy Access to m...,"Full use of living room, kitchen (with dishwas...","English, spanish, german, russian, some french...",NaN,NaN,NaN,https://a0.muscache.com/im/pictures/683224/4cc...,NaN,13660,https://www.airbnb.com/users/show/13660,Simon,2009-04-16,"Madrid, Madrid, Spain","Gay couple, heterofriendly, enjoy having guest...",within a few hours,100%,NaN,f,https://a0.muscache.com/im/users/13660/profile...,https://a0.muscache.com/im/users/13660/profile...,Hispanoamérica,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Madrid, Comunidad de Madrid, Spain",Chamartín,Hispanoamérica,Chamartín,Madrid,Comunidad de Madrid,28016,Madrid,"Madrid, Spain",ES,Spain,40.45628,-3.67763,t,Apartment,Private room,2,1.0,1.0,0.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Elevator,Heat...",172.0,$70.00,$350.00,$950.00,$0.00,$5.00,2,$15.00,1,365,1,1,365,365,1.0,365.0,today,t,29,59,89,89,2019-11-09,68,9,2010-03-14,2019-10-10,97.0,10.0,10.0,10.0,10.0,9.0,10.0,t,NaN,NaN,f,f,flexible,f,f,2,0,2,0,0.58,0.392050
1,21853,https://www.airbnb.com/rooms/21853,20191109094951,2019-11-09,Bright and airy room,We have a quiet and sunny room with a good vie...,I am living in a nice flat near the centre of ...,We have a quiet and sunny room with a good vie...,none,We live in a leafy neighbourhood with plenty o...,We are a 15 min bus ride away from the Casa de...,The flat is near the centre of Madrid (15 minu...,There is fibre optic internet connection for y...,If I am at home and see each other around here...,Many thanks for not smoking at home.,NaN,NaN,https://a0.muscache.com/im/pictures/68483181/8...,NaN,83531,https://www.airbnb.com/users/show/83531,Abdel,2010-02-21,"Madrid, Madrid, Spain",EN-ES-FR\r\nEN\r\nHi everybody: I'm Abdel. I'm...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/83531/profile...,https://a0.muscache.com/im/users/83531/profile...,A

Con esto ya tengo el dataset con las viviendas y se le ha añadido una columna con el sentiment que viene de los reviews

In [0]:
#Guardo el df_sentiment resultante
path_sentiment = ruta + 'data/listing_sentiment.csv'
listing_sentiment = house_data_sentimient.to_csv(path_sentiment, header=True)